1.	Import libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

2.	Load datasets

In [2]:
imdb_df = pd.read_csv("../data/raw/netflix_movies_and_shows.csv")  # IMDb dataset
titles_df = pd.read_csv("../data/raw/netflix_titles.csv")           # Old dataset

3.	Basic info & shape

In [3]:
for name, df in [("IMDb", imdb_df), ("Titles", titles_df)]:
    print(f"\n{name} dataset")
    print("Shape:", df.shape)
    print(df.dtypes)


IMDb dataset
Shape: (5283, 11)
index                  int64
id                    object
title                 object
type                  object
description           object
release_year           int64
age_certification     object
runtime                int64
imdb_id               object
imdb_score           float64
imdb_votes           float64
dtype: object

Titles dataset
Shape: (8807, 12)
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


4. Missing values summary

In [4]:
for name, df in [("IMDb", imdb_df), ("Titles", titles_df)]:
    print(f"\n{name} missing values:")
    print(df.isna().sum())


IMDb missing values:
index                   0
id                      0
title                   0
type                    0
description             5
release_year            0
age_certification    2285
runtime                 0
imdb_id                 0
imdb_score              0
imdb_votes             16
dtype: int64

Titles missing values:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


5. Duplicates check

In [5]:
for name, df in [("IMDb", imdb_df), ("Titles", titles_df)]:
    print(f"\n{name} duplicates:", df.duplicated().sum())


IMDb duplicates: 0

Titles duplicates: 0


6. Overlap check

In [6]:
imdb_titles = set(imdb_df["title"].str.lower().str.strip())
titles_titles = set(titles_df["title"].str.lower().str.strip())
common_titles = imdb_titles & titles_titles
print(f"Common titles: {len(common_titles)}")
print(f"Unique to IMDb: {len(imdb_titles - common_titles)}")
print(f"Unique to Titles: {len(titles_titles - common_titles)}")

Common titles: 3911
Unique to IMDb: 1324
Unique to Titles: 4890


7. Example preview of common and unique

In [7]:
print("\nSample common titles:", list(common_titles)[:10])
print("\nSample unique IMDb:", list(imdb_titles - common_titles)[:10])
print("\nSample unique Titles:", list(titles_titles - common_titles)[:10])


Sample common titles: ['50m2', 'the delivery boy', 'muran', 'bareilly ki barfi', 'arq', 'thunder force', 'due date', 'one by two', 'abla fahita: drama queen', 'the outpost']

Sample unique IMDb: ['ok kanmani', 'kayko and kokosh', "nikki glaser: bangin'", 'dany boon: des hauts-de-france', 'lupin the third: the castle of cagliostro', 'the seven deadly sins: prisoners of the sky', 'the rite', 'the goop lab', 'dunkirk', 'dancing on glass']

Sample unique Titles: ['escape from mr. lemoncello’s library', '20 feet from stardom', 'oddbods: the festive menace', "the witness who didn't see anything", 'el patrón, radiografía de un crimen', 'rake', 'the duchess', 'chhota bheem & ganesh', 'ancient aliens', 'high on the hog: how african american cuisine transformed america']


8. Report

## What the side‑by‑side check tells us

	•	IMDb dataset (target present): 5,283 rows, clean, almost no missing except age_certification (~43%) and a few description/imdb_votes.
	•	Titles dataset (metadata only): 8,807 rows, rich fields (director, cast, country, rating, listed_in, duration) but no target.
	•	Overlap: 3,911 titles match by title (case/trim) alone. That means ~74% of IMDb rows can be enriched immediately with extra metadata; ~26% will remain IMDb-only unless we fuzzy match.

## Recommendation

	•	Use IMDb as the base (it has imdb_score, your target).
	•	Do a left join from IMDb → Titles on (title, release_year) after normalizing titles. This gives the best of both worlds without losing any IMDb rows.
	•	Optional later: try fuzzy matching for some of the 1,324 IMDb‑only titles to lift enrichment coverage (worth it only if the extra features are needed).


9. Normalize keys for a clean join

In [8]:
def norm_title(s: pd.Series) -> pd.Series:
    """Lowercase and strip whitespace for matching."""
    return s.str.strip().str.lower()

imdb_df["title_norm"] = norm_title(imdb_df["title"])
titles_df["title_norm"] = norm_title(titles_df["title"])

# Align join_year with release_year for strict match
titles_df["join_year"] = titles_df["release_year"]

# Metadata columns we want to pull from Titles dataset
meta_cols = ["director","cast","country","date_added","rating","duration","listed_in"]

10. Create unique tables

In [9]:
#Strict table unique on (title_norm, join_year)
titles_strict = titles_df[["title_norm", "join_year", *meta_cols]].copy()
dups_strict = titles_strict.duplicated(["title_norm", "join_year"]).sum()
if dups_strict:
    # keep the first occurrence; you can change policy if you want
    titles_strict = titles_strict.drop_duplicates(["title_norm", "join_year"], keep="first")

In [10]:
# Loose table unique on title_norm only
titles_loose = titles_df[["title_norm", *meta_cols]].copy()
dups_loose = titles_loose.duplicated(["title_norm"]).sum()
if dups_loose:
    titles_loose = titles_loose.drop_duplicates(["title_norm"], keep="first")

In [11]:
print(f"Strict duplicates removed: {dups_strict}")
print(f"Loose duplicates removed:  {dups_loose}")

Strict duplicates removed: 4
Loose duplicates removed:  6


11. Pass 1: Strict match

In [12]:
strict = imdb_df.merge(
    titles_strict,
    left_on=["title_norm","release_year"],
    right_on=["title_norm","join_year"],
    how="left",
    indicator=True
)

matched_strict = strict[strict["_merge"] == "both"].copy()
unmatched = strict[strict["_merge"] == "left_only"].copy()

# Drop merge indicators from matched
matched_strict.drop(columns=["_merge"], inplace=True)

# Drop failed merge columns from unmatched
unmatched.drop(columns=[
    c for c in unmatched.columns if c in meta_cols or c in ("join_year","_merge")
], inplace=True, errors="ignore")

print(f"IMDb rows total: {len(imdb_df)}")
print(f"Strict matches:   {len(matched_strict)}")
print(f"Strict unmatched: {len(unmatched)}")

IMDb rows total: 5283
Strict matches:   3074
Strict unmatched: 2209


12. Pass 2: Loose match for remaining

In [13]:
loose = unmatched.merge(
    titles_loose,
    on="title_norm",
    how="left",
    indicator=True
)

loose.drop(columns=["_merge"], inplace=True)

13. Combine strict + loose

In [14]:
combined = pd.concat([matched_strict, loose], ignore_index=True, sort=False)

# Drop join_year after merge
combined.drop(columns=["join_year"], inplace=True, errors="ignore")

# How many rows got *any* metadata
got_any_meta = combined[meta_cols].notna().any(axis=1).sum()
print(f"Rows enriched with at least one metadata field: "
      f"{got_any_meta} / {len(combined)} "
      f"({got_any_meta/len(combined):.1%})")

Rows enriched with at least one metadata field: 3956 / 5283 (74.9%)


14. Save outputs

In [15]:
Path("../data/processed").mkdir(parents=True, exist_ok=True)
Path("../data/interim").mkdir(parents=True, exist_ok=True)

strict_only_path = "../data/processed/netflix_merged_strict_only.csv"
combined_path    = "../data/processed/netflix_merged_strict_then_loose.csv"
unmatched_path   = "../data/interim/unmatched_after_loose.csv"

matched_strict.to_csv(strict_only_path, index=False)
combined.to_csv(combined_path, index=False)

still_unenriched = combined[combined[meta_cols].isna().all(axis=1)]
still_unenriched.to_csv(unmatched_path, index=False)

print("\nSaved files:")
print("-", strict_only_path)
print("-", combined_path)
print("-", unmatched_path)


Saved files:
- ../data/processed/netflix_merged_strict_only.csv
- ../data/processed/netflix_merged_strict_then_loose.csv
- ../data/interim/unmatched_after_loose.csv


15. Conclusion

What we’ve learned so far

	•	Rows in IMDb: 5,283
	•	Strict match (title + year): 3,074 rows (~58%)
	•	Strict + loose (title only): 3,956 rows with at least one metadata field (~75%)
	•	Still missing all metadata after both passes: ~25%

Interpretation:

	•	The Netflix Titles catalog only partially overlaps with the IMDb list (different availability/regions/years).
	•	The loose pass buys ~17% more rows with metadata, but also risks minor mis-matches (remakes, different years, same title different entities).
	•	For modeling quality, I’d prefer strict-only (clean labels). For EDA and the future app, the combined file is useful to show directors/genres/etc. on more rows.